# Cytoscape Jupyter Widget Demo

![](https://cytoscape.org/images/logo/cy3logoOrange.svg)

![](cyjupyter1.png)

By Keiichiro ONO (kono at ucsd dot edu)

Updated on 10/11/2018


## Introduction

This is a demo notebook for Cytoscape Jupyter Widget: a network visualizer for Jupyter Notebook.  It's a very simple graph visualizer using [Cytoscape.js](http://js.cytoscape.org).  You can interactively visualize graphs (networks) in CX or Cytoscape.js format.


## Requirments

This notebook was tested with the following:

* cyjupyter v0.2.0
* Jupyter Notebook v5.6
* NetworkX v2.2


### Official Web Site and Code Repository

* https://github.com/idekerlab/cytoscape-jupyter-widget 


## Install Widget

### From PyPi

```bash
pip install cyjupyter
jupyter nbextension enable --py --sys-prefix cyjupyter
```


### From Source

```bash
git clone　https://github.com/idekerlab/cytoscape-jupyter-widget.git
cd cytoscape-jupyter-widget
./build.sh
```

## How to Use the Widget

### 1. Import Widget

In [1]:
from cyjupyter import Cytoscape

### 2. Prepare Data

This widget supports graph data in CX or Cytoscape.js JSON (cyjs).

#### Cytoscape.js JSON

Cytoscape.js JSON is a simple format for graph data.  The following code generates Cytoscape.js data as a standard Python dictionary.  Yu can find the complete documentation about this format [here](http://js.cytoscape.org/#notation/elements-json)

In [4]:
# A Cytoscape.js network with 4 nodes and 5 edges represented as Python Dict

minimal_cyjs_network = {
    'elements': {
        'nodes': [
            {'data' : {'id': 'node 1'}},
            {'data' : {'id': 'node 2'}},
            {'data' : {'id': 'node 3'}},
            {'data' : {'id': 'node 4'}}
        ],
        'edges': [
            { 'data': {'id': 'edge1','source': 'node 1','target': 'node 2'}},
            { 'data': {'id': 'edge2','source': 'node 1','target': 'node 3'}},
            { 'data': {'id': 'edge3','source': 'node 2','target': 'node 3'}},
            { 'data': {'id': 'edge4','source': 'node 2','target': 'node 4'}},
            { 'data': {'id': 'edge5','source': 'node 3','target': 'node 4'}}
        ]
    }
}

#### CX JSON

CX is a JSON format used for network data transfer in the Cytoscape ecosystem.  You can download any network from [NDEx](http://www.ndexbio.org/) in CX format.  The following example uses NDEx Python client to download the network in CX.  You can change the _uuid_ to try any networks in NDEx.

In [2]:
# Python client for the NDEx database
import ndex2.client as nc

# Unique ID for a network entry in NDEx
uuid ='f28356ce-362d-11e5-8ac5-06603eb7f303'

# NDEx public server URL
ndex_url = 'http://public.ndexbio.org/'

# Create an instance of NDEx client
ndex=nc.Ndex2(ndex_url)

# Download the network in CX format
response=ndex.get_network_as_cx_stream(uuid)
print('Response code from NDEx: ', response.status_code)

# Store the data in a Python object
cx = response.json()

Response code from NDEx:  200


### Use Cytoscape Desktop for data conversion

[Cytoscape Desktop](http://manual.cytoscape.org/en/stable/Supported_Network_File_Formats.html#cytoscape-js-json) supports import/export data in CX and Cytoscape.js JSON.  Please read the document to learn more about the feature. 

### 3. Render it!

#### Cytoscape.js Networks

By default, the widget is configured to display networks in Cytoscape.js JSON.  If you pass pass it as _*data*_ parameter, it automatically render it using default _Visual Style_ and [cose layout](http://js.cytoscape.org/#layouts/cose).  Since Cytoscape.js JSON is the default data format for this Widget, you don't need to pass _format_ option.

In [5]:
Cytoscape(data=minimal_cyjs_network)

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'node 1'}}, {'data': {'id': 'node 2'}}, {'data': {'id':…

#### CX Networks

To display CX networks, you just need to pass the List object (CX is always a list of _aspects_ for more information about CX, please read the [official document](http://www.home.ndexbio.org/data-model/)) **AND SPECIFY FORMAT, cx**.  If the data contains Visual Style and layout, it will be applied automatically.  Otherwise, default ones will be used.

In [6]:
Cytoscape(data=cx, format='cx')

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'consistencyGroup': …

### Use files exported from Cytoscape

You can easily use files edited in [Cytoscape Desktop](https://cytoscape.org/) because it supports both CX and Cytoscpae.js formats by default.  In the following example, it will display a [KEGG pathway diagram](https://www.genome.jp/kegg-bin/show_pathway?org_name=hsa&mapno=00020&mapscale=&show_description=hide) imported into Cytoscape Desktop using [KEGGScape](http://apps.cytoscape.org/apps/keggscape) and exported to Cytoscape.json files.

(Network object and Visual Style are exported to separate files.)

In [7]:
import json

# Network file exported from Cytoscape (in .cyjs format)
with open('hsa00020.cyjs') as f:
    kegg = json.load(f)

# Style file exported from Cytoscape (in Cytoscape.js Style JSON format)
with open('kegg-styles.json') as f2:
    kegg_style = json.load(f2)

# In this case, actual style is a first element in the array...
style_obj = kegg_style[0]['style']
    
# Pass the style and network to the widget (layout is optional)
Cytoscape(data=kegg, visual_style=style_obj, layout={'height': '700px'})

Cytoscape(data={'format_version': '1.0', 'generated_by': 'cytoscape-3.6.1', 'target_cytoscapejs_version': '~2.…

## Optional Parameters

You can use most of the basic Cytoscape.js features through optional parameters.

### Cell Height

This widget uses [standard Layout mechanism for Jupyter Widgets](https://minrk-ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-layout-attribute.).  To specify the cell height, you can pass it as a part of _layout_ parameter:

In [8]:
Cytoscape(data=cx, format='cx', layout={'height': '300px'})

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'consistencyGroup': …

### Automatic Graph Layout

If there is no **layout_name** parameter, default layout (cose layout algorithm) will be used.  To specify one of the layout algorithms available in Cytoscape.js, just pass the name.

#### Preset Layout

If you have already specified (x, y) positions for nodes, the parameter will be used as the node positions.  However, if you pass this **layout_name** parameter, it will be override those original positions.

In [9]:
Cytoscape(data=cx, format='cx', layout_name='grid')

Cytoscape(data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': [{'consistencyGroup': …

### Custom _Visual Style_

This widget fully supports Cytoscape.js compatible styls.  Just pass your style as a dict.  For more information about styling, please visit Cytoscape.js official web site to learn how to create data mappings:

* Cytoscape.js [Style](http://js.cytoscape.org/#style)

Also, you can change background color by passing _background_ parameter as string.

In [15]:
# Custom visual style as Python Dict
my_style = [
    {
        'selector': 'node',
        'style': {
            'background-color': '#EFEFEF',
            'label': 'data(name)',
            'width': 10,
            'height': 10,
            'shape': 'diamond',
            'color': '#EEEEEE',
            'font-weight': 400,
            'text-halign': 'right',
            'text-valign': 'bottom',
            'font-size': 12
        }
    },
    {
        'selector': 'edge',
        'style': {
            'width': 3,
            'opacity': 0.8,
            'line-color': 'green',
            'target-arrow-color': 'green',
            'target-arrow-shape': 'triangle'
        }
    }
]

Cytoscape(data=cx, format='cx', layout_name='breadthfirst', visual_style=my_style, background='#000000')

Cytoscape(background='#000000', data=[{'numberVerification': [{'longNumber': 281474976710655}]}, {'metaData': …

## Use with Other Tools and Libraries

This widget is a simple network visualizer, but is powerful if you use with other popular libraries.

### Network X

[NetworkX](https://networkx.github.io/) is a widely used library for network analysis.  It has basic visualization features, but you can use advanced visualization feature of Cytoscape.js using this widget.

**NetworkX v2.x series has I/O library for Cytoscape.js objects, and we strongly recommend to use v2.2 or newer.**

#### Example 1: Use NetworkX as a randome graph generator

NetworkX has several randome graph generators and you can easily use the data in CyJupyter:

In [16]:
# You must install NetworkX before running the following cells!
import networkx as nx
from networkx.readwrite import cytoscape_data
from networkx.algorithms import betweenness_centrality, edge_betweenness_centrality

In [17]:
network1 = nx.scale_free_graph(100)
cyjs1 = cytoscape_data(network1)
Cytoscape(data=cyjs1)

Cytoscape(data={'data': [], 'directed': True, 'multigraph': True, 'elements': {'nodes': [{'data': {'id': '0', …

#### Example 2: Calculate network statistics and use it in visualization

NetworkX has basic visualization features, but you can replace that part with this widget.

In [33]:
G = nx.waxman_graph(150)

# Calculate betweenness centrarity of the graph (for both nodes and edges)
bc = betweenness_centrality(G)
ebc = edge_betweenness_centrality(G)
nx.set_node_attributes(G, bc, 'betweenness')
nx.set_edge_attributes(G, ebc, 'betweenness')

# Pick min and max for each data
bc_min = min(bc.values())
bc_max = max(bc.values())
ebc_min = min(ebc.values())
ebc_max = max(ebc.values())

# Create visual mappings

# For nodes
mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', 5, 50)'
op_mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', 0.3, 1)'
color_mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', #EFEFEF, #00897B)'

# For edges
edge_mapper = 'mapData(betweenness,' + str(ebc_min) + ',' + str(ebc_max) + ', 1, 10)'
edge_op_mapper = 'mapData(betweenness,' + str(ebc_min) + ',' + str(ebc_max) + ', 0.3, 1)'

# Build a style
custom_visual_style = [
    {
        'selector': 'node',
        'style': {
            'background-color': color_mapper,
            'width': mapper,
            'height': mapper,
            'background-opacity': op_mapper,
            'label': "data(id)"
        }
    },
    {
        'selector': 'edge',
        'style': {
            'width': edge_mapper,
            'opacity': edge_op_mapper,
            'line-color': '#333333'        
        }
    }
]

# Convert to Cytoscape.js JSON
cyjs2 = cytoscape_data(G)

# Visualize it
Cytoscape(data=cyjs2,  layout_name='circle', visual_style=custom_visual_style, background='#EEEEEE')

Cytoscape(background='#EEEEEE', data={'data': [], 'directed': False, 'multigraph': False, 'elements': {'nodes'…

#### Example 3: Use NetworkX as layout engine

NetworkX has a nice drawing features, but all generated images are static.  If you use it as a layout engine, you can visualize it using the node positions calculated in NetworkX (or graphviz, which is supported by NetworkX):

In [36]:
# Layout using NetworkX
pos = nx.kamada_kawai_layout(G)
nodes = cyjs2['elements']['nodes']

SCALE_FACTOR = 700 # This adjust density of the view, because original positions are too dense...

# Assign (X, Y) positions
for node in nodes:
    node_id = node['data']['id']
    node['position'] = {
        'x': pos[int(node_id)][0] * SCALE_FACTOR,
        'y': pos[int(node_id)][1] * SCALE_FACTOR
    }

# Render it with new positions
Cytoscape(data=cyjs2, visual_style=custom_visual_style)

Cytoscape(data={'data': [], 'directed': False, 'multigraph': False, 'elements': {'nodes': [{'data': {'pos': (0…